# Imbalanced binary classification and custom metrics

Данильченко Вадим

In [1]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
import numpy as np
import pandas as pd
from tqdm import trange
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


создадим класс, вычисляющий основные метрики качетсва классификации

In [2]:
class Calculate_metrics:
    
    def __init__(self, y_true, y_pred):
        self.y_true = y_true
        self.y_pred = y_pred
        self.cm = self.confusion_matrix()

    # функция считает матрицу ошибок
    def confusion_matrix(self):
        cm = pd.DataFrame(index=['pred_true', 'pred_false'], columns=['true', 'false'], data=0)

        for i in trange(len(self.y_true)):
            if self.y_true[i]==1:
                if self.y_pred[i]==1:
                    cm.loc['pred_true', 'true'] += 1
                else:
                    cm.loc['pred_false', 'true'] += 1
            elif self.y_true[i]==0:
                if self.y_pred[i]==1:
                    cm.loc['pred_true', 'false'] += 1
                else:
                    cm.loc['pred_false', 'false'] += 1
        print('confusion matrix ready')
        return cm
    
    # функция возвращает метрику precision(точность)
    def precision(self):
        return self.cm.loc['pred_true', 'true']/(self.cm.loc['pred_true', 'true'] + self.cm.loc['pred_true', 'false'])

    # функция возвращает метрику recall(полнота)
    def recall(self):
        return self.cm.loc['pred_true', 'true']/(self.cm.loc['pred_true', 'true'] + self.cm.loc['pred_false', 'true'])
    
    # возвращает значение f1-меры
    def f1_score(self):
        precision = self.precision()
        recall = self.recall()
        return 2 * ((precision * recall) / (precision + recall))

    # точность
    def accuracy_score(self):
        return sum((1 if self.y_true[i]==self.y_pred[i] else 0) for i,_ in enumerate(self.y_pred))/len(self.y_pred)

посмотрим на результаты классификации на исходных данных

In [3]:
X, y = make_classification(n_classes=2, class_sep=2,
    weights=[0.05, 0.95], n_informative=3, n_redundant=1, flip_y=0,
    n_features=20, n_clusters_per_class=1, n_samples=1000, random_state=10)
print('Original dataset shape %s' % Counter(y))

Original dataset shape Counter({1: 950, 0: 50})


In [4]:
# разделим данные на тренировочную и тестовую выборки
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=777)

In [5]:
# обучим классификатор
clf = DecisionTreeClassifier(random_state=777)
clf.fit(x_train, y_train)
y_pred = clf.predict_proba(x_test)

In [6]:
# выведем метрики
print('результат классификации по исходным данным')
metrics = Calculate_metrics(y_test, y_pred[:,1])
print(metrics.cm, '\n{:-^50}'.format(''))
print('precision: ', metrics.precision(),
     '\nrecall: ', metrics.recall(),
     '\nf1_score: ', metrics.f1_score(),
     '\naccuracy_score: ', metrics.accuracy_score(),
     )

результат классификации по исходным данным


100%|██████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 3399.02it/s]


confusion matrix ready
            true  false
pred_true    193      0
pred_false     1      6 
--------------------------------------------------
precision:  1.0 
recall:  0.9948453608247423 
f1_score:  0.9974160206718347 
accuracy_score:  0.995


посмотрим на результат классфикации после увеличения миноритарного класса

In [7]:
# аугментируем целевой класс
ros = RandomOverSampler(random_state=42)
X_res, y_res = ros.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_res))

Resampled dataset shape Counter({0: 950, 1: 950})


In [8]:
x_train, x_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=777)
clf = DecisionTreeClassifier(random_state=777)
clf.fit(x_train, y_train)
y_pred = clf.predict_proba(x_test)

In [9]:
# выведем метрики
print('результат классификации после аугментации')
metrics = Calculate_metrics(y_test, y_pred[:,1])
print(metrics.cm, '\n{:-^50}'.format(''))
print('precision: ', metrics.precision(),
     '\nrecall: ', metrics.recall(),
     '\nf1_score: ', metrics.f1_score(),
     '\naccuracy_score: ', metrics.accuracy_score(),
     )

результат классификации после аугментации


100%|██████████████████████████████████████████████████████████████████████████████| 380/380 [00:00<00:00, 3699.21it/s]


confusion matrix ready
            true  false
pred_true    193      0
pred_false     0    187 
--------------------------------------------------
precision:  1.0 
recall:  1.0 
f1_score:  1.0 
accuracy_score:  1.0
